# Lecture 1 Notes

#### Computational Models
* Using computation to help understand the world in which we live
* Experimental devices that help us understand something that has happened or predict the future

    - Optimization models
    - Statistical models
    - Simulation models

##### What is an Optimization Model?
* An objective function that is to be maximized or minimized
    e.g. Minimize time spent traveling from New York to Boston
* A set of constraints (possibly empty) that must be honored
    e.g. Cannot spend more than $100.00
         Must be in Boston before 1700

#### **Takeaways**
* Many problems of real importance can be formulated as an optimization problem
* Reducing a seemingly new problem to an instance of a well-known problem allows one to use pre-existing methods for solving them.
* Solving optimization problems is computationally challenging
* A greedy algorithm is often a practical approach to finding a pretty good approximation solution to an optimization problem

# Chapter 14.1 Knapsack and Graph Optimization problems
What is the algorithm efficiency of greedy?
    - The time complexity of the built-in sorted function
    - The number of elements in items
        i.e. The number of iterations of the loop is &#920;(n), where _n_ is the length of items
            The worst time for Pythons built-in sorting function is &#920;(n log n) where _n_ is he length of the list to be sorted






We have decided that an approximation is not good enough i.e., we want the best possible solution to the 0/1 Knapsack Problem

$$\sum_{i=0}^{n-1} V[i] \times I[i].\text{value}$$

* Each item is represented by a pair <value, weight>
* The knapsack can accomodate items with a total weight of no more than _w_.
* A vector, _I_, of length _n_, represents the set of available items. Each element of the vector is an item.
* A vector, _V_, of length _n_, is used to indicate whether or not each item iss taken by the burglar. If _V[i]_ = 1, item _I[i]_ is taken. If _V[i]_ = 0, item _I[i]_ is not taken.
* Find a _V_ that maximizes

In [19]:
class Item(object):
    def __init__(self, n, w, v):
        self._name = n
        self._value = v
        self._weight = w

    def get_name(self):
        return self._name

    def get_value(self):
        return self._value

    def get_weight(self):
        return self._weight

    def __str__(self):
        return '\n{:>15}: <value: {:>3}, weight: {:>3}>'.format(self._name, self._value, self._weight)

    __repr__ = __str__

def build_items():
    names = ['clock', 'painting', 'radio', 'vase', 'book', 'computer']
    values = [175, 90, 20, 50, 10, 200]
    weights = [10, 9, 4, 2, 1, 20]
    items = []
    for i in range(len(values)):
        items.append(Item(names[i], weights[i], values[i]))
    return items

def get_binary_rep(n, num_digits):
    """
    Assumes n and num_digits are non-negative integers
    Returns a str of length num_digits that is a binary representation of n.
    """
    result = ''
    while n > 0:
        result = str(n % 2) + result
        n = n//2
    if len(result) > num_digits:
        raise ValueError('noe enough digits')
    return '0' * (num_digits - len(result)) + result

def gen_powerset(items, constraint, get_val, get_weight):
    """
    Yield all subsets (as lists of items) whose total weight <= constraint.
    Early-prunes branches that already exceed the constraint.
    """
    n = len(items)

    def backtrack(idx, cur_subset, cur_wt):
        # if overweight, abandon this branch
        if cur_wt > constraint:
            return
        # if we've considered all items, emit the feasible subset
        if idx == n:
            yield list(cur_subset)
            return

        # 1) skip current item
        yield from backtrack(idx + 1, cur_subset, cur_wt)

        # 2) take current item (if it still fits)
        it = items[idx]
        cur_subset.append(it)
        yield from backtrack(idx + 1, cur_subset, cur_wt + get_weight(it))
        cur_subset.pop()

    yield from backtrack(0, [], 0.0)


def choose_best(pset, max_weight, get_val, get_weight):
    best_val = 0.0
    best_set = []

    for items in pset:
        total_value = 0.0
        total_weight = 0.0
        valid = True

        for item in items:
            total_weight += get_weight(item)
            if total_weight > max_weight:
                valid = False
                break  # 💡 stop adding items—constraint exceeded
            total_value += get_val(item)

        if valid and total_value > best_val:
            best_val = total_value
            best_set = items

    return best_set, best_val


def test_best(max_weight=20):
    items = build_items()
    feasible = list(gen_powerset(items, max_weight, Item.get_value, Item.get_weight))
    best_set = max(feasible, key=lambda S: sum(Item.get_value(x) for x in S))
    best_val =sum(Item.get_value(x) for x in best_set)
    return best_set, best_val


In [20]:
taken, val = test_best()
print('Total value of items taken is ', val)
for item in taken:
    print(item)

Total value of items taken is  275

          clock: <value: 175, weight:  10>

       painting: <value:  90, weight:   9>

           book: <value:  10, weight:   1>


The complexity of this implementation is order _$\Theta(n\cdot 2^n)$_, where _n_ is the length of items.

The function gen_powerset returns a list of lists of Items. This list is of length _$2^n$_, and the longest list in it is of length n. Therefore the outer loop in choose_best will be executed _$\Theta(2^n)$_ times, and and then number of times the inner loop will be executed is bounded by _n_.